In [45]:
from datetime import datetime as dt
from arcgis.gis import GIS
from arcgis.geoanalytics.analyze_patterns import forest, glr
gis = GIS('https://ndhagsb01.esri.com/portal', 
          'admin', 
          'esri.agp2', 
          profile="your_enterprise_portal", verify_cert=False)

In [46]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime as dt

import arcgis
import arcgis.geoanalytics
from arcgis.gis import GIS

from arcgis.geoanalytics.manage_data import run_python_script

In [47]:
import arcgis
arcgis.env.verbose = True

In [48]:
search_result = gis.content.search("", item_type = "big data file share", max_items=40)
search_result

[<Item title:"bigDataFileShares_air_quality_2019" type:Big Data File Share owner:admin>,
 <Item title:"bigDataFileShares_pm2017" type:Big Data File Share owner:admin>,
 <Item title:"bigDataFileShares_pm2017_4326" type:Big Data File Share owner:admin>]

In [49]:
air_item = search_result[0]

In [50]:
air_lyr = air_item.layers[0]

In [51]:
air_lyr

<Layer url:"https://ndhga01.esri.com/gis/rest/services/DataStoreCatalogs/bigDataFileShares_air_quality_2019/BigDataCatalogServer/air_quality">

In [54]:
def code():
    from datetime import datetime as dt
    import pyspark.sql.functions as F
    from pyspark.sql.functions import concat, col, lit
    from datetime import datetime as dt
    df = layers[0]
    cols = ['Site Num', 'County Code', 'State Code', 'Date Local', 'Time Local', 
            'Parameter Name', 'Sample Measurement', 'Latitude', 'Longitude']
    df = df.select(cols)
    df = df.withColumn('Site_Num', F.lpad(df['Site Num'], 4, '0'))
    df = df.withColumn('County_Code', F.lpad(df['County Code'], 3, '0'))
    df = df.withColumn('State_Code', F.lpad(df['State Code'], 2, '0'))
    df = df.withColumn('unique_id', F.concat(F.col('State_Code'), F.col('County_Code'), F.col('Site_Num')))
    id = int(df.select('unique_id').first()[0])
    df = df.where(col("unique_id") == id)
    
    drop_cols = ['Site_Num', 'County_Code', 'State_Code', 'Site Num', 'County Code', 'State Code']
    df = df.drop('Site_Num', 'County_Code', 'State_Code', 'Site Num', 'County Code', 'State Code')
    df = df.withColumn('datetime', concat(col("Date Local"), lit(" "), col("Time Local")))
    drop_cols = ['Time Local', 'Date Local']
    df = df.drop('Time Local', 'Date Local')
    
    df_pm25 = df.filter(df['Parameter Name'] == 'PM2.5 - Local Conditions')
    df_pm10 = df.filter(df['Parameter Name'] == 'PM10 - LC')
    df_windspeed = df.filter(df['Parameter Name'] == 'Wind Speed - Resultant')
    df_windir = df.filter(df['Parameter Name'] == 'Wind Direction - Resultant')
    
    df_pm25 = df_pm25.withColumnRenamed('Sample Measurement', df_pm25.select('Parameter Name').first()[0])
    df_pm25 = df_pm25.drop('Parameter Name')

    df_pm10 = df_pm10.withColumnRenamed('Sample Measurement', df_pm10.select('Parameter Name').first()[0])
    df_pm10 = df_pm10.drop('Parameter Name')

    df_windspeed = df_windspeed.withColumnRenamed('Sample Measurement', df_windspeed.select('Parameter Name').first()[0])
    df_windspeed = df_windspeed.drop('Parameter Name')

    df_windir = df_windir.withColumnRenamed('Sample Measurement', df_windir.select('Parameter Name').first()[0])
    df_windir = df_windir.drop('Parameter Name')
    
    new_df1 = df_pm25.join(df_pm10, on=['datetime'], how='inner')
    new_df2 = df_windir.join(df_windspeed, on=['datetime'], how='inner')
    new_df3 = new_df2.join(new_df1, on=['datetime'], how='inner')
    
    new_df3.write.format("webgis").save("fbmodeldata" + str(dt.now().microsecond))

In [55]:
run_python_script(code=code, layers=[air_lyr])

Submitted.
Executing...
Executing (RunPythonScript): RunPythonScript "def code():\n    from datetime import datetime as dt\n    import pyspark.sql.functions as F\n    from pyspark.sql.functions import concat, col, lit\n    from datetime import datetime as dt\n    df = layers[0]\n    cols = ['Site Num', 'County Code', 'State Code', 'Date Local', 'Time Local', \n            'Parameter Name', 'Sample Measurement', 'Latitude', 'Longitude']\n    df = df.select(cols)\n    df = df.withColumn('Site_Num', F.lpad(df['Site Num'], 4, '0'))\n    df = df.withColumn('County_Code', F.lpad(df['County Code'], 3, '0'))\n    df = df.withColumn('State_Code', F.lpad(df['State Code'], 2, '0'))\n    df = df.withColumn('unique_id', F.concat(F.col('State_Code'), F.col('County_Code'), F.col('Site_Num')))\n    id = int(df.select('unique_id').first()[0])\n    df = df.where(col("unique_id") == id)\n    \n    drop_cols = ['Site_Num', 'County_Code', 'State_Code', 'Site Num', 'County Code', 'State Code']\n    df = df.

{"messageCode":"BD_101029","message":"287/522 distributed tasks completed.","params":{"completedTasks":"287","totalTasks":"522"}}
{"messageCode":"BD_101029","message":"308/522 distributed tasks completed.","params":{"completedTasks":"308","totalTasks":"522"}}
{"messageCode":"BD_101029","message":"327/522 distributed tasks completed.","params":{"completedTasks":"327","totalTasks":"522"}}
{"messageCode":"BD_101029","message":"344/522 distributed tasks completed.","params":{"completedTasks":"344","totalTasks":"522"}}
{"messageCode":"BD_101029","message":"365/522 distributed tasks completed.","params":{"completedTasks":"365","totalTasks":"522"}}
{"messageCode":"BD_101029","message":"383/522 distributed tasks completed.","params":{"completedTasks":"383","totalTasks":"522"}}
{"messageCode":"BD_101029","message":"401/522 distributed tasks completed.","params":{"completedTasks":"401","totalTasks":"522"}}
{"messageCode":"BD_101029","message":"418/522 distributed tasks completed.","params":{"com

{"messageCode":"BD_101138","message":"[Python] TypeError: 'NoneType' object is not subscriptable","params":{"text":"TypeError: 'NoneType' object is not subscriptable"}}


{"messageCode":"BD_101139","message":"An error while executing the Python script."}


{"messageCode":"BD_101139","message":"An error while executing the Python script."}


Failed to execute (RunPythonScript).


Failed to execute (RunPythonScript).


Failed at Sat Mar 28 12:11:14 2020 (Elapsed Time: 9 minutes 20 seconds)
Failed.


Failed.


Exception: Job failed.

In [36]:
item1 = gis.content.search('df_pm25', 'feature layer')[0]

In [37]:
item2 = gis.content.search('df_temp', 'feature layer')[0]

In [38]:
item3 = gis.content.search('df_pm10', 'feature layer')[0]

In [10]:
item = gis.content.search('point_data_air_quality', 'feature layer')[0]

In [11]:
item

<Item title:"point_data_air_quality313441" type:Table Layer owner:admin>

In [13]:
df = item.tables[0].query(as_df=True)

In [15]:
df.columns

Index(['Barometric_pressure', 'Latitude', 'Latitude1', 'Latitude2',
       'Latitude3', 'Latitude4', 'Latitude5', 'Latitude6', 'Longitude',
       'Longitude1', 'Longitude2', 'Longitude3', 'Longitude4', 'Longitude5',
       'Longitude6', 'OBJECTID', 'Outdoor_Temperature', 'PM10___LC',
       'PM2_5___Local_Conditions', 'Relative_Humidity',
       'Wind_Direction___Resultant', 'Wind_Speed___Resultant', 'datetime',
       'globalid', 'unique_id'],
      dtype='object')

In [16]:
df = df[['Barometric_pressure', 'Latitude', 'Longitude', 'Outdoor_Temperature', 'PM10___LC',
       'PM2_5___Local_Conditions', 'Relative_Humidity',
       'Wind_Direction___Resultant', 'Wind_Speed___Resultant', 'datetime',
       'globalid', 'unique_id']]

In [18]:
df.to_csv('AirQualityData')

In [21]:
from arcgis.features import GeoAccessor

In [22]:
sdf = GeoAccessor.from_xy(df, 'Longitude', 'Latitude')

C:\Users\pri10421.AVWORLD\.conda\envs\tf-py-fast-arcgis\lib\site-packages\arcgis\features\geo\_io\fileops.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SHAPE'] = geoms
C:\Users\pri10421.AVWORLD\.conda\envs\tf-py-fast-arcgis\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\pri10421.AVWORLD\.conda\envs\tf-py-fast-arcgis\lib\site-packages\arcgis\features\geo\_accessor.py:1021: SettingWithCopyWarning: 
A value is trying to be set 

In [33]:
sdf.dtypes

Barometric_pressure            float64
Latitude                       float64
Longitude                      float64
Outdoor_Temperature            float64
PM10___LC                      float64
PM2_5___Local_Conditions       float64
Relative_Humidity              float64
Wind_Direction___Resultant     float64
Wind_Speed___Resultant         float64
datetime                        object
globalid                        object
unique_id                       object
SHAPE                         geometry
dtype: object

In [24]:
data = gis.content.import_data(sdf, 'air_quality_fl')

In [31]:
data

<Item title:"cc38faf8abd4407b9f735361024906f1" type:Feature Layer Collection owner:admin>

In [26]:
df = data.layers[0].query(as_df=True)

In [27]:
df

,SHAPE,barometric,datetime,fid,globalid,latitude,longitude,outdoor_te,pm10___lc,pm2_5___lo,relative_h,unique_id,wind_direc,wind_speed
0,"{""x"": -13326772.361100657, ""y"": 4413970.097713...",999.5,2018-02-10 10:00,1,{6EFB48BA-4ED6-E70C-5C39-4BFDBB5445D9},36.819449,-119.716433,62.0,44.6,30.1,57.0,060195001,282.3,3.9
1,"{""x"": -13326772.361100657, ""y"": 4413970.097713...",1008.0,2018-02-21 07:00,2,{721F4790-B0D6-6A19-3BF0-430AEBF629A0},36.819449,-119.716433,34.0,5.1,15.3,70.0,060195001,167.1,0.9
2,"{""x"": -13326772.361100657, ""y"": 4413970.097713...",1000.4,2018-02-18 09:00,3,{42A94F87-86D2-E7BA-9FCC-D81206AECF47},36.819449,-119.716433,56.0,0.4,15.7,44.0,060195001,273.7,3.7
3,"{""x"": -13326772.361100657, ""y"": 4413970.097713...",1010.0,2018-02-08 07:00,4,{3FD7F79D-3A61-DE97-0866-57DFAD866787},36.819449,-119.716433,47.0,55.6,23.0,83.0,060195001,91.5,1.6
4,"{""x"": -13326772.361100657, ""y"": 4413970.097713...",1005.9,2018-02-09 00:00,5,{86EC21C3-4035-8FF7-473E-9B5C8D7D97D7},36.819449,-119.716433,53.0,43.0,26.3,77.0,060195001,80.6,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663490,"{""x"": -9865256.95000663, ""y"": 5383189.96017645...",1006.0,2019-01-29 06:00,663491,{90B0BE4B-F659-F196-A73E-F7264B32131B},43.466111,-88.621111,-2.0,11.3,6.6,78.0,550270001,280.0,9.8
663491,"{""x"": -9865256.95000663, ""y"": 5383189.96017645...",1023.0,2019-01-31 05:00,663492,{6B1F9982-14F6-1F7B-EB50-CAA4D60C7D91},43.466111,-88.621111,-26.0,14.7,7.1,70.0,550270001,247.0,4.6
663492,"{""x"": -9865256.95000663, ""y"": 5383189.96017645...",1013.0,2019-02-17 22:00,663493,{D6A9BE01-C26F-4A35-6191-C8A4FADFC1E8},43.466111,-88.621111,19.0,7.7,4.6,90.0,550270001,8.0,7.3
663493,"{""x"": -9865256.95000663, ""y"": 5383189.96017645...",1023.0,2019-02-18 09:00,663494,{6E055DB8-06D9-2D3C-628B-1D42428BF868},43.466111,-88.621111,17.0,6.5,4.1,71.0,550270001,12.0,7.4


In [29]:
m1 = gis.map('USA')
m1

MapView(layout=Layout(height='400px', width='100%'))

In [30]:
m1.add_layer(data.layers[0])